In [ ]:
#Master en Ciencias Actuariales de la Universidad Rey Juan Carlos
#Asignatura: Renta variable y derivados financieros
#Práctica 1: bonos sintéticos y forward

#instalamos la librería numpy-financial
!pip install numpy-financial

#importamos la librería
import numpy_financial as npf

#en este ejercicio nos dan los precios de adquisición y flujos de cada para los bonos A, B, C, D, E
#para otro bono, X, tenemos los flujos de caja pero no el precio
#en base a los datos de los bonos A-E tenemos que calcular el precio de adquisición de X
#también tenemos que calcular la rentabilidad (TIR) de cada bono

#datos de los bonos
#rellenamos con 0 para que todos los vectores tengan la misma longitud
#es necesario incluir el desembolso inicial (precio del bono) como flujo de caja
#para luego poder calcular la TIR
bonoA = [-96,100,0,0,0,0]
bonoB = [-545,50,550,0,0,0]
bonoC = [-1030,70,70,1070,0,0]
bonoD = [-2030,150,150,150,2150,0]
bonoE = [-3920,300,300,300,300,4300]

#para el bono X, desconocemos el desembolso inicial así que de momento ahí ponemos 0
bonoX = [0,100,100,100,100,1100]

#el ejercicio de un bono será el opuesto del desembolso
precioA = -bonoA[0]
precioB = -bonoB[0]
precioC = -bonoC[0]
precioD = -bonoD[0]
precioE = -bonoE[0]

#calculamos la TIR de todos los bonos salvo X
tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
tirC = npf.irr(bonoC)
tirD = npf.irr(bonoD)
tirE = npf.irr(bonoE)

#comprobamos el cálculo
print(tirA)

#para calcular el precio del bono X tenemos que conocer la ETTI desde i=1 hasta 5
#la ETTI de un año i es:
#((flujo de caja en el año i para el bono que vence ese año /
# (valor actual de ese bono – valor actualizado de los cupones anteriores a i)) ^ (1/i) – 1

#así que, para calcular la ETTI del año i, necesitaremos el factor de descuento para el año i – 1
#El factor de descuento para un año i a su vez se calcula con la ETTI de ese año: V[i] = 1 / (1 + ETTI[i]))^i
#la clave es que para el año 1, ETTI = tasa de interés para ese año
#además, para un bono con vencimiento de un año, tasa de interés = TIR
#así que ETTI[0] = tasa de interés[0] = TIR del bono que tenga vencimiento a un año
#(en Python el primer elemento de un vector es el 0, no el 1)
etti = [tirA,0,0,0,0]


#definimos la función que calcula la ETTI para cualquier año y bono que deseemos
#FCID significa: flujos de caja intermedios descontados
#es decir, el valor actual de los cupones de cada bono
def calcular_etti(bono, etti):
    n = [i for i, e in enumerate(bono) if e != 0][-1]
    ultimo_flujo = bono[n] #flujo de caja para un bono en el año en que vence
    precio = -bono[0]
    fcid = 0
    for i in range(1,n):
        fcid += bono[i] / (1+etti[i-1])**i #esto es lo que antes llamamos “valor actualizado de los cupones anteriores a i”
    return (ultimo_flujo / (precio - fcid))**(1/n)-1
#respecto a esta última línea: el precio de un bono es su valor actual, es lo mismo

#cuidado que en la última línea de la función usamos “n” en lugar de i, pero eso es porque…
#cada bono tiene un vencimiento distinto. Hay un n para cada valor de i
#lo importante es que para el cálculo tenemos que elevar a: 1/1, 1/2, 1/3, etc
#NO tenemos que coger un n fijo para todos los años


#habiendo definido la función, calculamos los valores de ETTI desde i = 2 hasta 5
#recuerda que en Python el primer elemento de un vector es 0
#así que etti[1] es la ETTI para el 2º año
etti[1] = calcular_etti(bonoB, etti)
etti[2] = calcular_etti(bonoC, etti)
etti[3] = calcular_etti(bonoD, etti)
etti[4] = calcular_etti(bonoE, etti)

#creamos una función que calcule el precio de un bono con tipo de interés no constante
#recuerda que la "i" no representa el tipo de interés, sino la fila del vector, el año del bono
def calcular_precio(flujos, interés):
    precio = 0
    for i in range(1, len(flujos)):
        precio += flujos[i] * (1+interés[i-1])**-i #descontamos los flujos
    return precio

#calculamos precio del bono X
precioX = calcular_precio(bonoX, etti)

#en el vector que representa los flujos de caja del bono X añadimos el desembolso inicial
bonoX[0] = -precioX

#calculamos la TIR del bono X
tirX = npf.irr(bonoX)